There are several ways to read a ptu file and numeral teams have their own golden child of a python module to read it. One of the few I will be using fot references includes:

1. snAPI by quCoa
2. readPTU, opensourced by Jfeatherstone (Heriott Watt University)
3. readPTU_FLIM, opensourced by SumeetRohilla

In [ ]:
from snAPI.Main import *

# sn = snAPI()
# sn.getFileDevice(r"snAPI\local_code\antibunch.sptw\down_conv22.ptu")

# f = open("down_conv22.ptu", "r")
# print(f.read())

SBuf = "down_conv22.ptu".encode('utf-8')

conf = (SBuf * 65535)()
conf = str(conf, "utf-8").replace('\x00','')
print(conf)


In [1]:
from snAPI.Main import *
import matplotlib
matplotlib.use('TkAgg',force=True)
from matplotlib import pyplot as plt
print("Switched to:",matplotlib.get_backend())
import time

if(__name__ == "__main__"):

    sn = snAPI()
    sn.getFileDevice(r"down_conv22.ptu")

    #sn.getDeviceIDs()
    #sn.getDevice()
    sn.initDevice(MeasMode.T2)
    
    # set the configuration for your device type
    sn.loadIniConfig("config\TH260N.ini")
    
    # 1. shift the signals to max correlation max at tau = 0
    #sn.device.setInputChannelOffset(1, 1588)
    
    # 2. set windowSize and startTime
    sn.correlation.setG2Parameters(1, 2, 50000, 5)
    sn.correlation.measure(1000,savePTU=False)

    while True:
        finished = sn.correlation.isFinished()
        data, bins = sn.correlation.getG2Data()
        time.sleep(.3)
        
        plt.clf()
        plt.plot(bins, data, linewidth=2.0, label='g(2)')
        plt.xlabel('Time [s]')
        plt.ylabel('g(2)')
        plt.legend()
        plt.title("g(2)")
        plt.pause(0.1)
        
        if finished:
            break
    
    plt.show(block=True)

<>:18: SyntaxWarning: invalid escape sequence '\T'
<>:18: SyntaxWarning: invalid escape sequence '\T'
C:\Users\andre\AppData\Local\Temp\ipykernel_23140\614433316.py:18: SyntaxWarning: invalid escape sequence '\T'
  sn.loadIniConfig("config\TH260N.ini")


Switched to: TkAgg


In [6]:
from snAPI.Main import *
import matplotlib
matplotlib.use('TkAgg',force=True)
from matplotlib import pyplot as plt
print("Switched to:",matplotlib.get_backend())
import time

if(__name__ == "__main__"):

    sn = snAPI()
    sn.getFileDevice(r"down_conv22.ptu")
    sn.getMeasDescription()
    des_json = sn.measDescription
    print_json = json.dumps(sn.measDescription, indent=2)
    aqc_time = des_json["StopAfter"]

    print("Measurement Description: ", print_json)
    print("Acquisition Time: ", aqc_time)

    #sn.getDeviceIDs()
    #sn.getDevice()
    sn.initDevice(MeasMode.T2)
    
    # set the configuration for your device type
    sn.loadIniConfig("config\TH260N.ini")
    
    # 2. set windowSize and startTime
    sn.correlation.setG2Parameters(startChannel = 1, stopChannel = 2, windowSize = 2*aqc_time, binWidth = 5)
    #sn.correlation.setG2Parameters(1, 2, 50000, 5)
    sn.correlation.measure(1000,savePTU=False)
    
    while True:
        finished = sn.correlation.isFinished()
        data, bins = sn.correlation.getG2Data()
        time.sleep(.3)
        
        plt.clf()
        plt.plot(bins, data, linewidth=2.0, label='g(2)')
        plt.xlabel('Time [s]')
        plt.ylabel('g(2)')
        plt.legend()
        plt.title("g(2)")
        plt.pause(0.1)
        
        if finished:
            break
    
    plt.show(block=True)

<>:25: SyntaxWarning: invalid escape sequence '\T'
<>:25: SyntaxWarning: invalid escape sequence '\T'
C:\Users\andre\AppData\Local\Temp\ipykernel_17764\2405375970.py:25: SyntaxWarning: invalid escape sequence '\T'
  sn.loadIniConfig("config\TH260N.ini")


Switched to: TkAgg
Measurement Description:  {
  "AcqTime": 360000000,
  "AveSyncRate": 0.0,
  "AveSyncPeriod": 4294967295.0,
  "StopReason": "Manual",
  "StopAfter": 3030703.0,
  "WarningsFlag": 0,
  "NumRecs": 3596366
}
Acquisition Time:  3030703.0


In [ ]:
from snAPI.Main import *
import matplotlib
matplotlib.use('TkAgg',force=True)
from matplotlib import pyplot as plt
print("Switched to:",matplotlib.get_backend())
import time

if(__name__ == "__main__"):

    sn = snAPI()
    sn.getFileDevice(r"down_conv22.ptu")

    #sn.getDeviceIDs()
    #sn.getDevice()
    sn.initDevice(MeasMode.T2)
    
    # set the configuration for your device type
    sn.loadIniConfig("config\TH260N.ini")
    
    windowSize = 2000 #ps
    # move the histograms of the both channels to the same time after the sync to filter both at once
    sn.device.setInputChannelOffset(0,-4000)

    # measure the count rate before the herald filter
    CrInIdx = sn.manipulators.countrate()
    
    # define a gate window after the herald channel 0 (sync) for the detector channels 1 and 2, 
    # starting at 52000 ps after the herald signal , with a gate length of windowSize (300 ps)
    heraldChans = sn.manipulators.herald(0, [1,2], 46000, windowSize, inverted=False, keepChannels=True )
    
    # measure the count rate after the herald filter
    CrOutIdx = sn.manipulators.countrate()

    # initiate the g2 correlation with the time-gated channels
    sn.correlation.setG2Parameters(heraldChans[0], heraldChans[1], windowSize, 1)
    sn.correlation.measure(10000,savePTU=False)

    while True:
        finished = sn.correlation.isFinished()
        data, bins = sn.correlation.getG2Data()
        
        CRin = sn.manipulators.getCountrates(CrInIdx)
        CRout = sn.manipulators.getCountrates(CrOutIdx)
        
        # print the count rates
        sn.logPrint(f"CR in: {CRin[1]}, {CRin[2]} - out: {CRout[heraldChans[0]]}, {CRout[heraldChans[1]]}")
        
        time.sleep(.3)
        
        plt.clf()
        plt.plot(bins, data, linewidth=2.0, label='g(2)')
        plt.xlabel('Time [s]')
        plt.ylabel('g(2)')
        plt.legend()
        plt.title("g(2)")
        plt.pause(0.1)
        
        if finished:
            break
    
    plt.show(block=True)